### Experiments to find the best evolutionary parameters to optimize the impact of synergistic nudges

In [28]:
import json
import numpy as np
from scipy import stats

import probability_distributions
import maximum_nudges
import evolutionary_algorithms as ea
import maximum_synergistic_nudge_evolutionary as synergistic_nudge_ev
import get_data

In [33]:
NUDGE_SIZE = 0.01
PATH_TO_DISTRIBUTIONS = "/home/joboti/azumi_derkjan/master_thesis/code/"

#### Define the input and output distributions for synergistic nudges

In [34]:
PERCENTAGE_MAX_ENTROPY_SIZE = 75
NUMBER_OF_VARS = 4
NUMBER_OF_STATES = 3
FILENAME_FORMAT_INPUT = "input_dist_{}.npy"
FILENAME_FORMAT_OUTPUT = "cond_output_dist_{}.npy"
DIST_START = 100
DIST_END = 120

distribution_numbers = list(range(DIST_START, DIST_END, 1))
path_to_limited_entropy_system_dists = (
    PATH_TO_DISTRIBUTIONS + "system_distributions/" 
    + "limited_entropy/"
)
input_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_INPUT, 
    distribution_numbers
)
input_dists = [input_dist for input_dist in input_dists]
output_dists = get_data.get_system_distributions_limited_entropy(
    path_to_limited_entropy_system_dists, PERCENTAGE_MAX_ENTROPY_SIZE,
    NUMBER_OF_VARS, NUMBER_OF_STATES, FILENAME_FORMAT_OUTPUT, 
    distribution_numbers
)
output_dists = [output_dist for output_dist in output_dists]

#### Define the evolutionary parameters

In [45]:
evolutionary_parameters = {
    "number_of_generations": 150,
    "population_size": 10,
    "number_of_children": 20, 
    "generational": True,
    "parent_selection_mode": "rank_exponential",
    "start_mutation_size": 0.01/25,
    "change_mutation_size": 0.01/(25*10),
    "mutations_per_update_step": 15
}



In [46]:
impacts = []
backup_filename = "find_params_synergistic_{}var_{}states_{}entropy_dists{}_{}.json".format(
    NUMBER_OF_VARS, NUMBER_OF_STATES, PERCENTAGE_MAX_ENTROPY_SIZE, DIST_START, DIST_END 
)
for input_dist, cond_output in zip(input_dists, output_dists):
    max_synergistic_nudge = synergistic_nudge_ev.find_synergistic_nudge_with_max_impact(
        input_dist, cond_output, NUDGE_SIZE, evolutionary_parameters
    )
    impact = max_synergistic_nudge.score
    new_distribution = max_synergistic_nudge.new_distribution
    l1_norm_to_old_distribution = np.sum(np.absolute(
        input_dist-max_synergistic_nudge.new_distribution
    ))
    if abs(l1_norm_to_old_distribution-NUDGE_SIZE) > 10**(-4):
        print("WARNING size of nudge {}".format(l1_norm_to_old_distribution))

    print("the max nudge impact {}".format(impact))
    impacts.append(impact)
    with open(backup_filename, 'w') as f:
        json.dump(impacts, f, indent=4)

print(np.mean(impacts))
print(np.std(impacts))
    

the max nudge impact -0.0061103394637
the max nudge impact -0.00799831021129
the max nudge impact -0.00593301722579
the max nudge impact -0.00776066975344
the max nudge impact -0.00668405291854
the max nudge impact -0.00747068725045
the max nudge impact -0.00693620862032
the max nudge impact -0.00720075015693
the max nudge impact -0.00746057378017
the max nudge impact -0.0071732080644
the max nudge impact -0.00700421984984
the max nudge impact -0.00599980231516
the max nudge impact -0.00766275674237
the max nudge impact -0.00748942198807
the max nudge impact -0.00701606697454
the max nudge impact -0.00559104804457
the max nudge impact -0.00662449999576
the max nudge impact -0.00713642617693
WARNING size of nudge 0.00980752346033
the max nudge impact -0.0072608635652
the max nudge impact -0.00646461536262
-0.006948876923
0.000641859912949
